In [ ]:
import pandas as pd
train=pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
train.head()

In [ ]:
train.nunique().sort_values(ascending=False)

In [ ]:
train.shape

In [ ]:
train=train.drop(['row_id'],axis=1)
train.head()

In [ ]:
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)

In [ ]:
train.time=pd.to_datetime(train.time, infer_datetime_format=True)
train.info()

In [ ]:
!pip install fast_ml
!pip install lazypredict

In [ ]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()
dt_fe.fit(train, datetime_variables=['time'])
train = dt_fe.transform(train)
train.head()

In [ ]:
train.nunique().sort_values(ascending=False)

In [ ]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

In [ ]:
train=train.drop(remove_col,axis=1)
train['time:is_weekend']=train['time:is_weekend'].replace({0:False,1:True})
train.head()

In [ ]:
train=train.drop(['time','time:time'],axis=1)
train.info()

In [ ]:
train['time:day_part'].unique()

In [ ]:
train['time:day_part'].value_counts()

In [ ]:
import numpy as np
train['time:day_part'] = train['time:day_part'].fillna(value=np.nan)
train['time:day_part'].unique()

In [ ]:
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)

In [ ]:
train[train.isnull().any(axis=1)]

In [ ]:
train['time:day_part'] = train['time:day_part'].fillna(value='midnight')
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)[:5]

In [ ]:
train['time:day_part']=train['time:day_part'].replace({'midnight':0,'dawn':1, 'early morning':2, 'late morning':3, 'noon':4,
                                                       'afternoon':5,'evening':6, 'night':7 })
      
train['time:day_part'].value_counts()

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train=train.sample(n = len(train)//30)
train.shape

In [ ]:
y = train.pop('congestion')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from lazypredict.Supervised import LazyRegressor

In [ ]:
clf = LazyRegressor(verbose=0,predictions=True)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models